<a href="https://colab.research.google.com/github/woneuy01/AudioToText/blob/master/Reddit_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import requests
import json
import csv
import time
#import datetime

In [26]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=100&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [27]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [23]:
import pandas as pd
from datetime import datetime
#import datetime
from datetime import timezone
import seaborn as sns
#from youtube_data import youtube_search
sub = input('Enter a keyword: ')
searchdateafter = input("Enter search date publish after (yyyymmdd):")
searchdatebefore = input("Enter search date publish before (yyyymmdd):")

searchdateafter = datetime.strptime(searchdateafter, "%Y%m%d")
after=int(searchdateafter.replace(tzinfo=timezone.utc).timestamp())
#searchdateafter=str(searchdateafter)+"T00:00:00Z"
print(after)

searchdatebefore = datetime.strptime(searchdatebefore, "%Y%m%d")
before=int(searchdatebefore.replace(tzinfo=timezone.utc).timestamp())
#searchdatebefore=str(searchdatebefore)+"T00:00:00Z"
print(before)

#query = ""
# the time should set to period 5/26--> published after 5/26 to published before 5/27

#test = youtube_search(keyword,searchdateafter,searchdatebefore)
#test.keys()

Enter a keyword: coronavirus
Enter search date publish after (yyyymmdd):20200601
Enter search date publish before (yyyymmdd):20200602
1590969600
1591056000


In [ ]:
from datetime import timezone
dt = datetime(2015, 10, 19)
timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
print(timestamp)

In [29]:
query = ""
subCount = 0
subStats = {}

data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    try:
      data = getPushshiftData(query, after, before, sub)
    except ValueError:
      print("response content is not valid json")
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590969600&before=1591056000&subreddit=coronavirus
100
2020-06-01 04:48:53
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1590986933&before=1591056000&subreddit=coronavirus
100
2020-06-01 11:02:25
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1591009345&before=1591056000&subreddit=coronavirus
100
2020-06-01 15:02:18
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1591023738&before=1591056000&subreddit=coronavirus
100
2020-06-01 18:23:58
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1591035838&before=1591056000&subreddit=coronavirus
100
2020-06-01 22:06:51
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1591049211&before=1591056000&subreddit=coronavirus
23
2020-06-01 23:54:11
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1591055651&before=1591056000&subreddit=c

In [30]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

523 submissions have added to list
1st entry is:
Arizona sees deadliest March and April on record created: 2020-06-01 00:05:44
Last entry is:
My favorite playlist during the corona lockdown. My advice against boredom 😃 created: 2020-06-01 23:54:11


In [ ]:
# def updateSubs_file():
#     upload_count = 0
#     location = ""
#     print("input filename of submission file, please add .csv")
    #filename = input()
    #file = location + filename
    # with open(file, 'w', newline='', encoding='utf-8') as file: 
    #     a = csv.writer(file, delimiter=',')
    #     headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
    #     a.writerow(headers)
    #     for sub in subStats:
    #         a.writerow(subStats[sub][0])
    #         upload_count+=1
               
        #print(str(upload_count) + " submissions have been uploaded")


In [31]:
posts=[]
for sub in subStats:
  posts.append(subStats[sub][0]) 
 
df=pd.DataFrame(posts, columns=["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"])
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,gua1j4,Arizona sees deadliest March and April on record,https://www.azfamily.com/news/arizona-sees-dea...,[deleted],0,2020-06-01 00:05:44,0,/r/Coronavirus/comments/gua1j4/arizona_sees_de...,USA
1,gua4or,"Brazil records 29,314 deaths (+480) and 514,84...",https://g1.globo.com/bemestar/coronavirus/noti...,EffectiveIndividual3,2,2020-06-01 00:10:58,4,/r/Coronavirus/comments/gua4or/brazil_records_...,Latin America
2,gua6kc,"Texas Tribune @TexasTribune Texas reported 1,9...",https://www.texastribune.org/2020/05/31/texas-...,[deleted],13,2020-06-01 00:14:06,13,/r/Coronavirus/comments/gua6kc/texas_tribune_t...,USA
3,guadd5,“Our calm is contagious”: How to use mindfulne...,https://www.vox.com/platform/amp/future-perfec...,shallah,1,2020-06-01 00:25:35,0,/r/Coronavirus/comments/guadd5/our_calm_is_con...,NaN
4,guagsl,"New coronavirus losing potency, top Italian do...",https://uk.reuters.com/article/us-health-coron...,[deleted],2,2020-06-01 00:31:32,2,/r/Coronavirus/comments/guagsl/new_coronavirus...,Good News


In [33]:
df.sort_values("Total No. of Comments", axis = 0, ascending = False, 
                 inplace = True, na_position ='last') 
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
290,gulu7b,57% of Americans say it’s more important to tr...,https://abcnews.go.com/Politics/hesitancy-resu...,HugeDetective0,36,2020-06-01 14:42:50,2375,/r/Coronavirus/comments/gulu7b/57_of_americans...,USA
65,gucr3h,‘Coronavirus Cooking Survey’ Finds That People...,https://www.goodnewsnetwork.org/people-are-eat...,Barknuckle,97,2020-06-01 03:05:04,688,/r/Coronavirus/comments/gucr3h/coronavirus_coo...,Good News
262,gul0hp,"Daily Discussion Post - June 01 | Questions, i...",https://www.reddit.com/r/Coronavirus/comments/...,AutoModerator,1,2020-06-01 13:52:39,684,/r/Coronavirus/comments/gul0hp/daily_discussio...,NaN
235,guk32n,The Protests Will Spread the Coronavirus,https://www.theatlantic.com/health/archive/202...,pump_the_brakes_son,1,2020-06-01 12:51:14,437,/r/Coronavirus/comments/guk32n/the_protests_wi...,USA
332,gun51t,Good news: the US saw a 21% drop in Covid deat...,https://coronachaser.com/?rows=year%2Ccountry%...,shreddit47,70,2020-06-01 15:58:27,414,/r/Coronavirus/comments/gun51t/good_news_the_u...,Good News


In [34]:
df=df[df["Total No. of Comments"]>300]
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
290,gulu7b,57% of Americans say it’s more important to tr...,https://abcnews.go.com/Politics/hesitancy-resu...,HugeDetective0,36,2020-06-01 14:42:50,2375,/r/Coronavirus/comments/gulu7b/57_of_americans...,USA
65,gucr3h,‘Coronavirus Cooking Survey’ Finds That People...,https://www.goodnewsnetwork.org/people-are-eat...,Barknuckle,97,2020-06-01 03:05:04,688,/r/Coronavirus/comments/gucr3h/coronavirus_coo...,Good News
262,gul0hp,"Daily Discussion Post - June 01 | Questions, i...",https://www.reddit.com/r/Coronavirus/comments/...,AutoModerator,1,2020-06-01 13:52:39,684,/r/Coronavirus/comments/gul0hp/daily_discussio...,NaN
235,guk32n,The Protests Will Spread the Coronavirus,https://www.theatlantic.com/health/archive/202...,pump_the_brakes_son,1,2020-06-01 12:51:14,437,/r/Coronavirus/comments/guk32n/the_protests_wi...,USA
332,gun51t,Good news: the US saw a 21% drop in Covid deat...,https://coronachaser.com/?rows=year%2Ccountry%...,shreddit47,70,2020-06-01 15:58:27,414,/r/Coronavirus/comments/gun51t/good_news_the_u...,Good News


In [35]:
#produce excel
df.to_csv(str(after)+".csv",encoding="utf-8-sig")

In [ ]:
gb7i90
https://api.pushshift.io/reddit/comment/search/?link_id=crzqmg&limit=1000&size=100&after='+str(after)+'&before='+str(before)
https://api.pushshift.io/reddit/search/comment/?q=trump&after=7d&aggs=created_utc&frequency=hour&size=0

In [68]:
def getPushshiftComments(PostId):
  url='https://api.pushshift.io/reddit/comment/search/?link_id='+str(PostId)+'&limit=100'
  print(url)
  r = requests.get(url)
  comments = json.loads(r.text)
  return comments['data']

In [69]:
subCount = 0
subStatsC = {}

In [70]:
def collectComments(subm):
    subData = list() #list to store data points
    permalink=subm['permalink']
    comment = subm['body']
    subreddit = subm['subreddit']
    subreddit_id = subm['subreddit_id']    
    total_awards_received = subm['total_awards_received']
    created = datetime.fromtimestamp(subm['created_utc']) #1520561700.0    
    
    subData.append((permalink,comment,subreddit,subreddit_id,total_awards_received,created))
    subStatsC[permalink] = subData

In [71]:
for link in df['Post ID']:
  try:
    data=getPushshiftComments(link)
    for submission in data:
      collectComments(submission)
      subCount+=1
  except:
    pass


https://api.pushshift.io/reddit/comment/search/?link_id=gulu7b&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=gucr3h&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=gul0hp&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=guk32n&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=gun51t&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=gulozi&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=guw0ot&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=gulgf2&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=guleez&limit=100
https://api.pushshift.io/reddit/comment/search/?link_id=gudz9j&limit=100


In [ ]:
k=0
i=0
for com in comments:
  leng=len(comments[k]['items'])
  for i in range(0,leng):
    #print(i)
    try:
      publishedAt.append(comments[k]['items'][i]['snippet']['topLevelComment']['snippet']['publishedAt'])
      videoId.append(comments[k]['items'][i]['snippet']['topLevelComment']['snippet']['videoId'])
      likeCount.append(comments[k]['items'][i]['snippet']['topLevelComment']['snippet']['likeCount'])
      textOriginal.append(comments[k]['items'][i]['snippet']['topLevelComment']['snippet']['textDisplay'])
      i+=1
    except:
      pass
  k+=1

         

In [ ]:
subStatsC

In [72]:
len(subStatsC)

1000

In [73]:
postsC=[]
for sub in subStatsC:
  postsC.append(subStatsC[sub][0]) 
 
dfc=pd.DataFrame(postsC, columns=["permalink","comment","subreddit","subreddit_id","total_awards_received","created"])
dfc.head(100)

,permalink,comment,subreddit,subreddit_id,total_awards_received,created
0,/r/Coronavirus/comments/gulu7b/57_of_americans...,Your post or comment has been removed because\...,Coronavirus,t5_2x4yx,0,2020-06-02 22:05:42
1,/r/Coronavirus/comments/gulu7b/57_of_americans...,Your post or comment has been removed because\...,Coronavirus,t5_2x4yx,0,2020-06-02 22:04:24
2,/r/Coronavirus/comments/gulu7b/57_of_americans...,Your post or comment has been removed because\...,Coronavirus,t5_2x4yx,0,2020-06-02 22:04:13
3,/r/Coronavirus/comments/gulu7b/57_of_americans...,Your post or comment has been removed because\...,Coronavirus,t5_2x4yx,0,2020-06-02 22:03:29
4,/r/Coronavirus/comments/gulu7b/57_of_americans...,Your post or comment has been removed because\...,Coronavirus,t5_2x4yx,0,2020-06-02 22:03:19
...,...,...,...,...,...,...
95,/r/Coronavirus/comments/gulu7b/57_of_americans...,"Yeah, because if the economy is utterly shut d...",Coronavirus,t5_2x4yx,0,2020-06-02 02:42:50
96,/r/Coronavirus/comments/gulu7b/57_of_americans...,To bad 50 percent of america is currently gath...,Coronavirus,t5_2x4yx,0,2020-06-02 02:38:50
97,/r/Coronavirus/comments/gulu7b/57_of_americans...,The woman who mentors me said it in April!! Wo...,Coronavirus,t5_2x4yx,0,2020-06-02 02:38:43
98,/r/Coronavirus/comments/gulu7b/57_of_americans...,[removed],Coronavirus,t5_2x4yx,0,2020-06-02 02:37:58


In [75]:
dfc.to_csv("reddit_comments"+str(searchdateafter)+"_"+str(searchdatebefore)+".csv",encoding="utf-8-sig")